<a href="https://colab.research.google.com/github/KoynaKarmakar/Skin-Cancer-Detection-Using-Deep-Learning/blob/main/Skin_Cancer_Resnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import zipfile
import os

# Define paths
zip_path = "/content/archive (1).zip"  # Path to the downloaded ZIP file
extract_path = "/content/skin_cancer_data"  # Destination folder for extracted files

# Extract dataset
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("✅ Dataset extracted successfully!")

# List extracted files & folders
os.listdir(extract_path)



✅ Dataset extracted successfully!


['ISIC_2024_Resize224']

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os

# List the contents of the extracted folder
extracted_files = os.listdir("/content/skin_cancer_data")
print("📂 Extracted files & folders:", extracted_files)


📂 Extracted files & folders: ['ISIC_2024_Resize224']


In [ ]:
import os

image_folder = "/content/skin_cancer_data/ISIC_2024_Resize224"

# List a few sample images
sample_images = os.listdir(image_folder)[:10]  # Show first 10 files
print("🖼 Sample image files:", sample_images)


🖼 Sample image files: ['train', 'test']


In [ ]:
import os

# Define the base dataset folder
image_folder = "/content/skin_cancer_data/ISIC_2024_Resize224"

# List subdirectories inside ISIC_2024_Resize224
subdirs = os.listdir(image_folder)
print("📂 Subdirectories:", subdirs)



📂 Subdirectories: ['train', 'test']


In [ ]:
import os

# Define paths
train_folder = "/content/skin_cancer_data/ISIC_2024_Resize224/train"
test_folder = "/content/skin_cancer_data/ISIC_2024_Resize224/test"

# List some images from the train folder
train_images = os.listdir(train_folder)[:10]  # Show first 10 images
print("🖼 Sample training images:", train_images)

# List some images from the test folder
test_images = os.listdir(test_folder)[:10]  # Show first 10 images
print("🖼 Sample test images:", test_images)


🖼 Sample training images: ['Malignant', 'Benign']
🖼 Sample test images: ['Malignant', 'Benign']


In [ ]:
import os

# Define paths
train_malignant = os.path.join(train_folder, "Malignant")
train_benign = os.path.join(train_folder, "Benign")

test_malignant = os.path.join(test_folder, "Malignant")
test_benign = os.path.join(test_folder, "Benign")

# List some images from each category
train_malignant_images = os.listdir(train_malignant)[:5]  # First 5 images
train_benign_images = os.listdir(train_benign)[:5]
test_malignant_images = os.listdir(test_malignant)[:5]
test_benign_images = os.listdir(test_benign)[:5]

print("🖼 Sample Malignant training images:", train_malignant_images)
print("🖼 Sample Benign training images:", train_benign_images)
print("🖼 Sample Malignant test images:", test_malignant_images)
print("🖼 Sample Benign test images:", test_benign_images)


🖼 Sample Malignant training images: ['4434.jpg', 'ISIC_0032687.jpg', '2500.jpg', '2032.jpg', '2590.jpg']
🖼 Sample Benign training images: ['4434.jpg', '2500.jpg', '2032.jpg', '2590.jpg', '2293.jpg']
🖼 Sample Malignant test images: ['6512.jpg', '5977.jpg', '6147.jpg', '5945.jpg', '6402.jpg']
🖼 Sample Benign test images: ['7263.jpg', '6512.jpg', '7250.jpg', '6682.jpg', '6402.jpg']


In [ ]:
import pandas as pd

# Create lists to store image paths and labels
image_paths = []
labels = []

# Function to load images from a folder
def load_images_from_folder(folder, label):
    for filename in os.listdir(folder):
        image_paths.append(os.path.join(folder, filename))
        labels.append(label)

# Load images from train and test folders
load_images_from_folder(train_malignant, "Malignant")
load_images_from_folder(train_benign, "Benign")
load_images_from_folder(test_malignant, "Malignant")
load_images_from_folder(test_benign, "Benign")

# Create a DataFrame
df_images = pd.DataFrame({"image_path": image_paths, "label": labels})

# Save CSV for reference
df_images.to_csv("/content/image_metadata.csv", index=False)

print("✅ Image metadata saved as image_metadata.csv")
print(df_images.head())


✅ Image metadata saved as image_metadata.csv
                                          image_path      label
0  /content/skin_cancer_data/ISIC_2024_Resize224/...  Malignant
1  /content/skin_cancer_data/ISIC_2024_Resize224/...  Malignant
2  /content/skin_cancer_data/ISIC_2024_Resize224/...  Malignant
3  /content/skin_cancer_data/ISIC_2024_Resize224/...  Malignant
4  /content/skin_cancer_data/ISIC_2024_Resize224/...  Malignant


In [ ]:
import tensorflow as tf
import pandas as pd

# Load metadata CSV
df_images = pd.read_csv("/content/image_metadata.csv")

# Image parameters
IMG_SIZE = (224, 224)
BATCH_SIZE = 32  # Adjust based on available memory

# Function to process images
def load_and_preprocess_image(image_path, label):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)  # Decode JPEG
    image = tf.image.resize(image, IMG_SIZE)  # Resize to (224, 224)
    image = image / 255.0  # Normalize to [0,1]
    return image, label

# Convert labels to binary values
df_images["label"] = df_images["label"].map({"Malignant": 1, "Benign": 0})

# Create TensorFlow dataset
image_paths = df_images["image_path"].values
labels = df_images["label"].values

dataset = tf.data.Dataset.from_tensor_slices((image_paths, labels))
dataset = dataset.map(load_and_preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)
dataset = dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

print("✅ Dataset created successfully!")




✅ Dataset created successfully!


In [ ]:
# Define split ratio
TRAIN_RATIO = 0.8

# Get dataset size
dataset_size = len(df_images)

# Calculate split index
train_size = int(TRAIN_RATIO * dataset_size)

# Shuffle dataset before splitting
dataset = dataset.shuffle(buffer_size=dataset_size, reshuffle_each_iteration=True)

# Split dataset
train_dataset = dataset.take(train_size)
val_dataset = dataset.skip(train_size)

train_dataset = train_dataset.prefetch(tf.data.AUTOTUNE)
val_dataset = val_dataset.prefetch(tf.data.AUTOTUNE)

print(f"✅ Training set size: {train_size}")
print(f"✅ Validation set size: {dataset_size - train_size}")


✅ Training set size: 11662
✅ Validation set size: 2916


In [ ]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras import layers, models

# Load ResNet50 without the top layer (pretrained on ImageNet)
base_model = ResNet50(weights="imagenet", include_top=False, input_shape=(224, 224, 3))

# Freeze the base model
base_model.trainable = False

# Build model
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(256, activation="relu"),
    layers.Dropout(0.3),
    layers.Dense(1, activation="sigmoid")  # Binary classification
])

# Compile model
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])



print("✅ ResNet model is ready!")


94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
✅ ResNet model is ready!


In [ ]:
# Define training parameters
# Train the model
history = model.fit(
    train_dataset,  # No need to apply .batch() again
    validation_data=val_dataset,
    epochs=10
)



print("✅ Model training complete!")


Epoch 1/10
456/456 ━━━━━━━━━━━━━━━━━━━━ 2661s 6s/step - accuracy: 0.4932 - loss: 0.8133
Epoch 2/10


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/epoch_iterator.py:151: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


456/456 ━━━━━━━━━━━━━━━━━━━━ 2575s 6s/step - accuracy: 0.4809 - loss: 0.6980
Epoch 3/10
456/456 ━━━━━━━━━━━━━━━━━━━━ 2600s 6s/step - accuracy: 0.5140 - loss: 0.6917
Epoch 4/10
456/456 ━━━━━━━━━━━━━━━━━━━━ 2609s 6s/step - accuracy: 0.4806 - loss: 0.6952
Epoch 5/10
456/456 ━━━━━━━━━━━━━━━━━━━━ 2612s 6s/step - accuracy: 0.4748 - loss: 0.6956
Epoch 6/10
456/456 ━━━━━━━━━━━━━━━━━━━━ 2522s 6s/step - accuracy: 0.4943 - loss: 0.6933
Epoch 7/10
456/456 ━━━━━━━━━━━━━━━━━━━━ 2565s 6s/step - accuracy: 0.5214 - loss: 0.6926
Epoch 8/10
456/456 ━━━━━━━━━━━━━━━━━━━━ 2522s 6s/step - accuracy: 0.5108 - loss: 0.6934
Epoch 9/10
456/456 ━━━━━━━━━━━━━━━━━━━━ 2512s 5s/step - accuracy: 0.5544 - loss: 0.6921
Epoch 10/10
456/456 ━━━━━━━━━━━━━━━━━━━━ 2557s 6s/step - accuracy: 0.5101 - loss: 0.6925
✅ Model training complete!
